In [1]:
import nltk
from nltk import word_tokenize
#nltk.download('punkt')
import math
import numpy as np
import csv
import random

[nltk_data] Downloading package punkt to /Users/yornvanda/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
# Read file
def readfile(fn):
  f = open(fn, 'r', encoding='utf-8')
  lines = [line for line in f.readlines()]
  f.close()
  return lines

# Read word from file
def readword(fn):
  f = open(fn, 'r', encoding='ISO-8859-1')
  lines = [line for line in f.readlines()]
  f.close()
  return lines

In [ ]:
readfile('positive-words.txt')

In [3]:
pos_reviews = readfile('positive-reviews.txt')
pos_words = readword('positive-words.txt')
neg_reviews = readfile('negative-reviews.txt')
neg_words = readword('negative-words.txt')

In [ ]:
pos_reviews = [w[:-1] for w in pos_reviews]
print(pos_reviews)

neg_reviews = [w[:-1] for w in neg_reviews]
print(neg_reviews)

for text in range(len(pos_reviews)):
  pos_reviews[text] = pos_reviews[text].lower()
print(pos_reviews)

for text in range(len(neg_reviews)):
  neg_reviews[text] = neg_reviews[text].lower()
print(neg_reviews)

pos_tokenized = [word_tokenize(i) for i in pos_reviews]
neg_tokenized = [word_tokenize(i) for i in neg_reviews]

In [ ]:
def feature_extract(tokeniz_review, positive_word, negative_word, label):
  _y = []
  _x1 = []
  _x2 = []
  _x3 = []
  _x4 = []
  _x5 = []
  _x6 = []

  if label == 'positive':
    y = 1
  else:
    y = 0
  for text in tokeniz_review:
    count_positive = 0
    count_negative = 0
    no = 0
    mark = 0
    pronoun =['i', 'me', 'my', 'you', 'your']
    count_pronoun = 0
    for word in text:
      if word in negative_word:
        count_negative += 1
      if word in positive_word:
        count_positive += 1
      if word == 'no':
        no = 1
      if word == '!':
        mark = 1
      if word in pronoun:
        count_pronoun += 1

    _y.append(y)
    _x1.append(count_positive)
    _x2.append(count_negative)
    _x3.append(no)
    _x4.append(count_pronoun)
    _x5.append(mark)

    if len(text) > 1:
      lenght = len(text)
      review_lenght = math.log(lenght)
    else:
      review_lenght = len(text)
    _x6.append(review_lenght)
  return _y, _x1, _x2, _x3, _x4, _x5, _x6

yp, x1p, x2p, x3p, x4p, x5p, x6p = feature_extract(pos_tokenized, pos_words, neg_words, 'positive')
print(len(x1p), len(yp))

print(yp[0])

yn, x1n, x2n, x3n, x4n, x5n, x6n = feature_extract(neg_tokenized, pos_words, neg_words, 'negative')
print(len(x1n), len(yn))

print(yn[0])

In [6]:
#Combine feature extraction together
y = yp + yn
x1 = x1p + x1n
x2 = x2p + x2n
x3 = x3p + x3n
x4 = x4p + x4n
x5 = x5p + x5n
x6 = x6p + x6n
print(len(y))

45868


In [ ]:
#field name
fn= ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'y']

rows = []
for data in range(len(y)):
  rows.append([x1[data], x2[data], x3[data], x4[data], x5[data], x6[data], y[data]])
# random.shuffle(rows)
print(rows)

# write to csv file
#with open('review.csv', 'w') as f:
  #writer = csv.writer(f)
  #writer.writerow(fn)
 # writer.writerows(rows)

In [ ]:
def read_from_csv(path):
  f = open(path, 'r')
  lines = f.readlines()
  f.close()
  lines.pop(0)
  
  x1 = []
  x2 = []
  x3 = []
  x4 = []
  x5 = []
  x6 = []
  y = []


  for data in lines:
    _x1, _x2, _x3, _x4, _x5, _x6, _y = data.split(',')
    x1.append(float(_x1))
    x2.append(float(_x2))
    x3.append(float(_x3))
    x4.append(float(_x4))
    x5.append(float(_x5))
    x6.append(float(_x6))
    y.append(float(_y))

  return x1, x2, x3, x4, x5 ,x6, y

path = 'review.csv'
x1, x2, x3, x4, x5, x6, y = read_from_csv(path)

tmp = list(zip(x1, x2, x3, x4, x5, x6, y))
random.shuffle(list(tmp))
print(tmp)
x1, x2, x3, x4, x5, x6, y = zip(*tmp)
x1 = list(x1)
x2 = list(x2)
x3 = list(x3)
x4 = list(x4)
x5 = list(x5)
x6 = list(x6)
y = list(y)

x1 = np.array(x1, dtype=np.float)
x2 = np.array(x2, dtype=np.float)
x3 = np.array(x3, dtype=np.float)
x4 = np.array(x4, dtype=np.float)
x5 = np.array(x5, dtype=np.float)
x6 = np.array(x6, dtype=np.float)
y = np.array(y, dtype = np.float)

x = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1), x4.reshape(-1, 1),
                    x5.reshape(-1, 1), x6.reshape(-1, 1)), axis=1)

In [10]:

from sklearn.model_selection import train_test_split
#training dataset
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=0) 

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [13]:
#Logistic Regression Model

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("Logistic Regression Accuracy Score -> ",accuracy_score(y_pred, y_test)*100 ,"%")


Logistic Regression Accuracy Score ->  60.45345541748419 %


In [14]:
#NB classifier model

model = naive_bayes.GaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("Naive Bayes Accuracy Score ->",accuracy_score(y_pred, y_test)*100 ,"%")


Naive Bayes Accuracy Score -> 55.297580117723996 %


In [15]:
#support vector machine model

model = svm.SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("Support Vector Machine Accuracy Score ->",accuracy_score(y_pred, y_test)*100, "%")


Support Vector Machine Accuracy Score -> 60.86766950076302 %


In [16]:
#KNN model

model = KNeighborsClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("KNN Accuracy Score ->",accuracy_score(y_pred, y_test)*100, "%")


KNN Accuracy Score -> 55.05777196424678 %


In [22]:
#gradient boosted model

model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("Gradient Boosted Accuracy Score ->",accuracy_score(y_pred, y_test)*100, "%")


Gradient Boosted Accuracy Score -> 60.638761717898404 %
